In [2]:
from selenium.webdriver.common.by import By
from selenium import webdriver
import time
import math
import pandas as pd
import numpy as np

##### You should specify the path to the chromedriver that is located in your computer and assign it to the path variable below.

In [3]:
url = 'https://en.bidfax.info/'
path = '/Users/areg/.wdm/drivers/chromedriver/mac64/108.0.5359/chromedriver'
# You should specify the path of the chromedriver in the brackets 
# that are in front of path variable above
driver = webdriver.Chrome(path)
driver.get(url)

/var/folders/7_/hrpf2wd52k91vfg1qd2m7ch80000gn/T/ipykernel_5311/3681283350.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


#### After calling select_make() function it will ask you to input a make of a car and then it will select it. If that make will not exit it will tell you to put the existing one.

In [4]:
def select_make():
    dropdown = driver.find_elements(By.XPATH, '//span[@class = "drop-down"]' )
    make = str(input('Please write the make you want to select:  '))
    def search_for_make(make):
        try:
            dropdown[0].click()
        except:
            pass
        try:
            make_ = driver.find_element(By.LINK_TEXT, make)
            make_.click()
            print(f"{make} make is selected successfully!")
        except:
            print(f'{make} make does not exist in this list. Please try again.')
            return select_make()
    search_for_make(make)

#### After calling select_model() fucntion it will ask you to input a model of a car and then it will select it. If that make will not exit it will tell you to put the existing one.

In [5]:
def select_model():
    dropdown = driver.find_elements(By.XPATH, '//span[@class = "drop-down"]' )
    model = str(input('Please write the model you want to select:  '))
    def search_for_model(model):
        try:
            dropdown[1].click()
        except:
            pass
        try:
            model_ = driver.find_element(By.LINK_TEXT, model)
            model_.click()
            print(f"{model} model is selected successfully!")
        except:
            print(f'{model} model does not exist in this list. Please try again.')
            return select_model()
    search_for_model(model)  

#### After calling select_year_start() function below it will ask you to input the starting year of a car you want to search for and then it will select it. If that make will not exit it will tell you to put the existing one. Call the select_year_start function after calling select_make() and select_model() functions

In [6]:
def select_year_start():
    def scrape_years():
        global years
        years_dropdown = driver.find_element(By.XPATH, '//div[@class = "filter_right"]')
        years = years_dropdown.find_elements(By.XPATH, ('./select'))
    scrape_years()
    start = str(input("Type in the starting year of the car:  "))
    def year_start(start):   
        year_start = years[0]
        try:
            year_start = year_start.find_elements(By.XPATH, ("./option"))
            for year in year_start:
                if year.text == start:
                    print(f"{start} year is selected successfully!")
                    return year.click()
            raise print(f"{start} year does not exist. Please try again.")
        except:
            return select_year_start()
    year_start(start)

#### After calling select_year_to() function it will ask you to input the ending year of a car you want to search for and then it will select it. If that make will not exit it will tell you to put the existing one.  Call the select_year_start function after calling select_make() and select_model() functions

In [7]:
def select_year_to():
    def scrape_years():
        global years
        years_dropdown = driver.find_element(By.XPATH, '//div[@class = "filter_right"]')
        years = years_dropdown.find_elements(By.XPATH, ('./select'))    
    scrape_years()
    to = str(input("Type in the ending year of the car:  "))
    def year_end(year_to):   
        year_to = years[1]
        try:
            year_to = year_to.find_elements(By.XPATH, ("./option"))
            for year in year_to:
                if year.text == to:
                    print(f"{to} year is selected successfully!")
                    return year.click()
            raise print(f"{to} year does not exist. Please try again.")
        except:
            return select_year_to()
    year_end(to)
    

#### The function scrape_page() scrape's the useful data from the website and store's it in lists. You don't have to call this function because it is called in the create_df() and add_next_pages() functions that are listed below.

In [8]:
def scrape_page():
    def scrape_data():
        global ln_dmg_dos_scraped; global ln_dmg_dos_scraped2; global prices_scraped
        prices_scraped = driver.find_elements(By.XPATH, '//span[@class = "prices"]')
        ln_dmg_dos_scraped = driver.find_elements(By.XPATH, '//p[@class = "short-story"]')
        ln_dmg_dos_scraped2 = driver.find_elements(By.XPATH, '//p[@class = "short-story2"]')
    def scrap_prices():
        global prices
        prices = []
        for price in prices_scraped:
            if price.text != '':
                prices.append(price.text)
    def scrape_lot_numbers():
        global lot_numbers
        lot_numbers = []
        for element in ln_dmg_dos_scraped:
            if 'Lot number' in element.text:
                lot_numbers.append(element.find_element(By.XPATH, 'span[@class = "blackfont"]').text)
    def scrape_conditions():
        global conditions
        conditions = []
        for element in ln_dmg_dos_scraped2:
            if 'Condition' in element.text:
                conditions.append(element.find_element(By.XPATH, 'span[@class = "blackfont"]').text)
    def scrape_damages():
        global damages
        damages = []
        for element in ln_dmg_dos_scraped:
            if 'Damage' in element.text:
                damages.append(element.find_element(By.XPATH, 'span[@class = "blackfont"]').text)
    def scrape_mileages():
        global mileages
        mileages = []
        for element in ln_dmg_dos_scraped2:
            if 'Mileage' in element.text:
                mileages.append(element.find_element(By.XPATH, 'span[@class = "blackfont"]').text)
    def scrape_dates_of_sale():
        global dates_of_sale
        dates_of_sale = []
        for element in ln_dmg_dos_scraped:
            if 'Date' in element.text:
                dates_of_sale.append(str(element.find_element(By.XPATH, 'span[@class = "blackfont"]').text))
    def dict_data():
        global columns_; global data_
        columns_ = ['lot_number', 'condition', 'damage', 'mileage', 'date_of_sale','price' ]
        data_ = [lot_numbers, conditions, damages, mileages, dates_of_sale, prices]
    scrape_data()
    scrape_conditions()
    scrap_prices()
    scrape_dates_of_sale()
    scrape_lot_numbers()
    scrape_mileages()
    scrape_damages()
    dict_data()

#### After calling create_df() function it will create a dictionary using data stored in lists created from scrape_page() function and then it will create a dataframe with the dictionary.

In [9]:
def create_df():
    time.sleep(3)
    scrape_page()
    df_data = {}
    for i, j in zip(columns_, data_):
        df_data[i]= j
    global df
    df = pd.DataFrame(df_data)


#### For calling the function add_next_page() you have to type in the number of pages you want to add to your dataframe that you created already by create_df() function. Call add_next_pages() function after calling create_df() function.

In [10]:
def add_next_pages(count):
    def next_page():
        next_page = driver.find_element(By.LINK_TEXT, '»')
        next_page.click()
    def add_to_df():
        scrape_page()
        df_data = {}
        global df
        for i, j in zip(columns_, data_):
            df_data[i] = j
        df1 = pd.DataFrame(df_data)
        df = df.append(df1)
        df = df.reset_index(drop = True)
    for i in range(int(count)):
        next_page()
        add_to_df()

#### After calling all functions above the date of cars in the dataframe will be in one column. The modify_dates() function separates the day, month and the year and creates separate columns with them.

In [11]:
def modify_dates():
    
    days = []
    for i in df['date_of_sale']:
        days.append(i[:2])
    df['day'] = days

    months = []
    for i in df['date_of_sale']:
        months.append(i[3:5])
    df['month'] = months
    
    years = []
    for i in df['date_of_sale']:
        years.append(i[6:])
    df['year'] = years
    df.drop('date_of_sale', axis = 'columns', inplace = True)
    df['day'].astype(dtype = int, copy = False)
    df['month'].astype(dtype = int, copy = False)
    df['year'].astype(dtype = int, copy = False)

In [12]:
select_make()

Volkswagen make is selected successfully!


In [13]:
select_model()

Gti model is selected successfully!


In [14]:
select_year_start()

2017 year is selected successfully!


In [15]:
select_year_to()

2020 year is selected successfully!


In [16]:
create_df()

In [17]:
#You can write any amount of pages between the 
#brackets of add_next_pages() function below
add_next_pages(3)

/var/folders/7_/hrpf2wd52k91vfg1qd2m7ch80000gn/T/ipykernel_5311/2061226291.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1)


In [18]:
modify_dates()
driver.close()

In [19]:
df.head()

,lot_number,condition,damage,mileage,price,day,month,year
0,73140642,Run And Drive,Minor Dent/Scratches,18405 miles (Actual),8900,27,01,2023
1,73369772,Engine Start Program,Front End,76675 miles (Actual),1150,26,01,2023
2,72185972,Run And Drive,Side,143759 miles (Actual),3200,26,01,2023
3,37939813,Run And Drive,Front End,58565 miles (Actual),6600,26,01,2023
4,73506952,Enhanced Vehicles,Front End,0 miles (Not Actual),650,25,01,2023


In [1]:
df.tail()

NameError: name 'df' is not defined